<a href="https://colab.research.google.com/github/michael-rowland/Pitch-Predictions/blob/master/Pitch-Predictions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## NOTES

---

QUESTIONS: 
- should I keep notes in docstring?

Features:
- Shifting?
- Home/Away

Per pitcher models

Exponential Smoothing

Permutation Importances

AVERAGE ROC AUC / SCORECARD

MULTIPLE PITCHERS

MODEL TYPES

## SETUP

---



In [8]:
%%capture
import sys

# If you're on Colab:
if 'google.colab' in sys.modules:
    !pip install category_encoders==2.*
    !pip install pybaseball

# If you're working locally:
else:
    DATA_PATH = '../data/'

In [97]:
# import os
# import numpy as np
import pandas as pd
import category_encoders as ce
from pybaseball import statcast_pitcher, playerid_lookup

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

## DATA CLEANING

---



In [11]:
def reorder(df):
  '''
  reorders pitches in ascending order
  '''
  df = df.copy()

  df = df.reindex(index=df.index[::-1])
  df = df.reset_index(drop=True)

  return df

In [174]:
def pitch_cleaning(df):
  '''
  cleans target ('pitch_type'), dropping NaN's (bad data) and non-standard 
  pitches
  '''
  df = df.copy()

  df = df.dropna(subset=['pitch_type'])
  
  df['pitch_type'] = df['pitch_type'].str.lower()
  remove_pitches = ['po', 'in', 'un']
  df = df[~df['pitch_type'].isin(remove_pitches)]
  df = df.drop('pitch_name', axis=1)

  return df

In [13]:
def historical_cleaning(df):
  '''
  shifts relevant historical data to prevent data leakage
  '''
  df = df.copy()

  df['previous_pitch'] = df['pitch_type'].shift(1) # target, should probably be dropped initially
  df['previous_strike'] = df['type'].shift(1) == 'S'
  df['previous_zone'] = df['zone'].shift(1) # this may need to be encoded somehow
  df = df.drop(['type', 'zone'], axis=1)

  return df

## DATA PROCESSING

---


In [14]:
def total_pitches(df):
  '''
  creates column of total pitches prior to current/target pitch
  '''
  df = df.copy()

  df['pitch_total'] = df.groupby('game_date').cumcount(ascending=True)

  return df

In [15]:
def score_margin(df):
  '''
  creates column of score margin for the pitcher's team
  '''
  df = df.copy()

  df['score_margin'] = df['fld_score'] - df['bat_score']
  df = df.drop(['fld_score', 'bat_score'], axis=1)

  return df

In [16]:
def pitcher_advantage(df):
  '''
  creates column of pitcher advantage based on the ball/strike count
  '''
  df = df.copy()

  advantages = {'0-0':'neutral', '0-1':'ahead', '0-2':'ahead', '1-0':'neutral', 
                '1-1':'neutral', '1-2':'ahead', '2-0':'behind', '2-1':'neutral',
                '2-2':'ahead', '3-0':'behind', '3-1':'behind', '3-2':'neutral'}

  df['count'] = df['balls'].astype(str) + '-' + df['strikes'].astype(str)
  df['pitcher_advantage'] = df['count'].map(advantages)
  df = df.drop('count', axis=1)

  return df

In [17]:
def baserunners_mapping(df):
  '''
  maps baserunner to boolean values and creates column if there is a baserunner 
  on any base
  '''
  df = df.copy()

  df['on_1b'] = df['on_1b'].notnull()
  df['on_2b'] = df['on_2b'].notnull()
  df['on_3b'] = df['on_3b'].notnull()
  
  df['baserunner'] = df['on_1b'] | df['on_2b'] | df['on_3b']

  return df

In [18]:
def at_bat_counter(df):
  '''
  creates column of number of batters faced
  '''
  df = df.copy()

  df['at_bat_count'] = df.groupby('at_bat_number').ngroup(ascending=True)+1
  df = df.drop('at_bat_number', axis=1)

  return df

In [19]:
def pitch_encoding(df):
  '''
  creates columns of previous pitch for each pitch type using OneHotEncoder
  '''
  df = df.copy()

  encoder = ce.OneHotEncoder(use_cat_names=True)
  encoded = encoder.fit_transform(df['previous_pitch'])
  df = pd.merge(df, encoded, how='outer', on=df.index)
  df = df.drop(['key_0', 'previous_pitch_nan'], axis=1)
  
  return df

In [20]:
def pitch_counter(df):
  '''
  creates columuns of cumulative count of each specific pitch type
  '''
  df = df.copy()

  pitches = [col for col in df if col.startswith('previous_pitch_')]
  
  for pitch in pitches:
    df[f'count_{pitch[-2:]}'] = (df[pitch] == 1).cumsum()

  return df

In [59]:
def pitch_tendency(df, window):
  '''
  creates columns of pitch tendency (mean) over last n = window pitches for 
  each pitch type
  '''
  df = df.copy()

  pitches = [col for col in df if col.startswith('previous_pitch_')]
  
  for pitch in pitches:
#     df[f'{pitch[-2:]}_tendency_{window}'] = df[pitch].iloc[1:].rolling(
    df[f'{pitch[-2:]}_tendency_{window}'] = df[pitch].rolling(
        window, min_periods=1).mean()
    
  return df

In [22]:
def strike_encoding(df):
  '''
  creates columns of strike pitch for each pitch type using OneHotEncoder
  '''
  df = df.copy()
  
  s = df['previous_strike'] == True
  df['strike_pitch'] = s.mask(s, df['previous_pitch'])
  
  encoder = ce.OneHotEncoder(use_cat_names=True)
  encoded = encoder.fit_transform(df['strike_pitch'])
  df = pd.merge(df, encoded, how='outer', on=df.index)
  df = df.drop(['key_0', 'strike_pitch_False', 'strike_pitch'], axis=1)
  
  return df

In [23]:
def strike_percentage(df, percentage=True):
  '''
  creates columns of strike percentage (or raw count) for each pitch type
  * This could be further engineered to calculate percentage of total number of 
  strikes thrown rather than number of total pitches
  '''
  df = df.copy()

  pitches = [col for col in df if col.startswith('strike_pitch_')]

  for i in pitches:
    if percentage:
      df[f'{i[-2:]}_cumulative_strikes'] = (df[f'strike_pitch_{i[-2:]}'] == 1).cumsum() / df['pitch_total']
    else:
      df[f'{i[-2:]}_cumulative_strikes'] = (df[f'strike_pitch_{i[-2:]}'] == 1).cumsum()
      
  return df

In [24]:
def strike_counter(df, window):
  '''
  creates columns of number of previous n (window) strikes thrown for each pitch 
  type
  '''
  df = df.copy()
  
  pitches = [col for col in df if col.startswith('strike_pitch_')]
  
  for i in pitches:
    target = f'{i[-2:]}_strike_tendency_{window}'
    df[target] = df[df[i] != 0][i].rolling(window, min_periods=1).sum()
    df[target] = df[target].fillna(method='ffill')
    df[target] = df[target].fillna(0)

  return df

In [113]:
def wrangle_game(df):
  '''
  this function does data cleaning and feature engineering on a per game basis
  '''
  df = df.copy()

  # CLEANING FUNCTIONS
  df = reorder(df)
  df = pitch_cleaning(df)
  if df.empty:
    return
  df = historical_cleaning(df)

  # FEATURE ENGINEERING FUNCTIONS
  df = total_pitches(df)
  df = score_margin(df)
  df = pitcher_advantage(df)
  df = baserunners_mapping(df)
  df = at_bat_counter(df)
  df = pitch_encoding(df)
  df = pitch_counter(df)
  
  windows = [5, 10, 20]
  for window in windows:
    df = pitch_tendency(df, window) 
  df = strike_encoding(df)
  df = strike_percentage(df, percentage=True)
  for window in windows:
    df = strike_counter(df, window)
  
  return df

## DATA GATHERING

---



In [194]:
playerid_lookup('berrios', 'jose')

Gathering player lookup table. This may take a moment.


,name_last,name_first,key_mlbam,key_retro,key_bbref,key_fangraphs,mlb_played_first,mlb_played_last
0,berrios,jose,621244,berrj001,berrijo01,14168,2016.0,2019.0


In [234]:
target = 'pitch_type'
features = ['pitch_name', 'pitch_type', 'game_date', 'stand', 'balls', 
            'strikes', 'on_3b', 'on_2b', 'on_1b', 'outs_when_up', 'inning', 
            'at_bat_number', 'bat_score', 'fld_score', 'type', 'zone']

In [235]:
def pitcher_wrangle(pitcher):
  '''
  gathers all available data on specific pitcher (prior to 2020 season)
  '''

  raw_data = statcast_pitcher('2008-04-01', '2020-01-01', pitcher)

  df = pd.DataFrame()

  games = raw_data['game_date'].unique().tolist()
  
  for game in games:
    game_data = wrangle_game(raw_data[raw_data['game_date'] == game][features])
    df = df.append(game_data, ignore_index=True, sort=False)
    
  return df

df = pitcher_wrangle(621244)

Gathering Player Data


## MODEL

---



In [236]:
# MOVE TO TOP WHEN FINISHED
from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import SimpleImputer
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from xgboost import XGBClassifier

In [179]:
def log_model(txt_file='model_log.txt'):
    with open(txt_file, 'a') as file:
        file.write(f'Validation accuracy: {model.score(X_val_processed, y_val):.5f}\n')
        file.write(f'Features: {X_train.columns}\n')
        file.write(f'{model.get_params}\n')
        file.write('\n')
        file.close()

In [197]:
train, test = train_test_split(df, train_size=0.8, random_state=42)
train, val = train_test_split(train, train_size=0.8, random_state=42)

In [198]:
target = 'pitch_type'
features = df.columns.drop([target, 'game_date'])

X_train = train[features]
y_train = train[target]
X_val = val[features]
y_val = val[target]
X_test = test[features]
y_test = test[target]

In [224]:
processor = make_pipeline(
    ce.OrdinalEncoder(),
    SimpleImputer(strategy='mean')
)
X_train_processed = processor.fit_transform(X_train)
X_val_processed = processor.transform(X_val)

model = RandomForestClassifier(
                               n_estimators=100, 
                               max_features=16,
                               max_depth=9, 
                               random_state=42, 
                               n_jobs=-1)
model.fit(X_train_processed, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=9, max_features=16,
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=-1, oob_score=False, random_state=42, verbose=0,
                       warm_start=False)

In [223]:
baseline = df['pitch_type'].value_counts(normalize=True)[0]
print(f'Baseline Accuracy: {baseline:.5f}')
print(f'Train Accuracy: {model.score(X_train_processed, y_train):.5f}')
print(f'Validation Accuracy: {model.score(X_val_processed, y_val):.5f}')

Baseline Accuracy: 0.32516
Train Accuracy: 0.65097
Validation Accuracy: 0.43076




---




In [260]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.feature_selection import f_regression, SelectKBest
from scipy.stats import randint, uniform

target = 'pitch_type'
features = df.columns.drop([target, 'game_date'])
train, test = train_test_split(df, train_size=0.8, random_state=0)
X_train = train[features]
y_train = train[target]
X_test = test[features]
y_test = test[target]

pipeline = make_pipeline(
    ce.OrdinalEncoder(), 
    SimpleImputer(), 
#     SelectKBest(f_regression), 
    RandomForestClassifier()
)

param_distributions = {
  'simpleimputer__strategy': ['mean', 'median'], 
#   'selectkbest__k': range(1, len(X_train.columns)+1), 
  'randomforestclassifier__n_estimators': randint(100, 200), 
#   'randomforestclassifier__max_features': range(1, 26), 
  'randomforestclassifier__max_features': uniform(0, 1), 
  'randomforestclassifier__max_depth': [5, 10, 15, 20, None], 
  'randomforestclassifier__min_samples_split': range(2, 11), 
}

search = RandomizedSearchCV(
  pipeline, 
  param_distributions=param_distributions, 
  n_iter=10, 
  cv=5, 
  scoring='accuracy', 
  verbose=10, 
  return_train_score=True, 
  n_jobs=-1
)

In [261]:
search.fit(X_train, y_train)

Fitting 5 folds for each of 10 candidates, totalling 50 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    6.2s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:   13.9s
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:   22.0s
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:   31.6s
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:   47.2s
[Parallel(n_jobs=-1)]: Done  41 out of  50 | elapsed:   56.1s remaining:   12.3s
[Parallel(n_jobs=-1)]: Done  47 out of  50 | elapsed:   58.8s remaining:    3.8s
[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed:  1.0min finished


RandomizedSearchCV(cv=5, error_score=nan,
                   estimator=Pipeline(memory=None,
                                      steps=[('ordinalencoder',
                                              OrdinalEncoder(cols=None,
                                                             drop_invariant=False,
                                                             handle_missing='value',
                                                             handle_unknown='value',
                                                             mapping=None,
                                                             return_df=True,
                                                             verbose=0)),
                                             ('simpleimputer',
                                              SimpleImputer(add_indicator=False,
                                                            copy=True,
                                                            fill_value=None,


In [262]:
search.best_params_

{'randomforestclassifier__max_depth': 10,
 'randomforestclassifier__max_features': 0.8871231883456001,
 'randomforestclassifier__min_samples_split': 8,
 'randomforestclassifier__n_estimators': 160,
 'simpleimputer__strategy': 'median'}

In [263]:
search.best_score_

0.4191224496948151

## MISC

---



In [ ]:
def pitch_map(df):
  '''
  generates dictionary mapping 'pitch_type' to 'pitch_name'
  '''
  df = df.copy()

  df = df.dropna(subset=['pitch_type', 'pitch_name'])
  pitch_types = df['pitch_name'].unique().tolist()
  pitch_types_abv = [i.lower() for i in df['pitch_type'].unique().tolist()]
  
  return dict(zip(pitch_types_abv, pitch_types))

# sample = 'https://raw.githubusercontent.com/michael-rowland/Pitch-Predictions/master/sample.csv'
# df = pd.read_csv(sample)
# df = df[df['game_date'] == '2018-09-28']